In [67]:
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

!pip install rasterio 
import rasterio as rio

!pip install imagecodecs
import tifffile

# Install TensorFlow Decision Forests.
!pip3 install tensorflow_decision_forests --upgrade
import tensorflow_decision_forests as tfdf

from sklearn.model_selection import train_test_split

from google.colab import drive
from google.colab import files
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Read Tiles Dataset**

In [48]:
tiles = pd.read_csv(r'/content/drive/MyDrive/capstone/Tiles_expanded.csv')

In [49]:
tiles.head(5)

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
tiles.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical'],
      dtype='object')

In [51]:
# x: 'Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
#    'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop', 'Collisions_Historical'
# y: 'Collisions_Future'

X = tiles[['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
           'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop', 'Collisions_Historical']]
y = tiles[['Collisions_Future']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.5)

In [52]:
IMAGE_PATH = '/content/drive/MyDrive/capstone/Satellite_Imagery/Satellite_Images_Tiled/' 

In [71]:
with rio.open("/content/drive/MyDrive/capstone/Satellite_Imagery/Satellite_Images_Tiled/36.tif") as img :

    img1= img.read()[0:148, 0:188, :]
    #imgmeta1=img.meta

with rio.open("/content/drive/MyDrive/capstone/Satellite_Imagery/Satellite_Images_Tiled/37.tif") as img :

    img2= img.read()[0:148, 0:188, :]
    #imgmeta2=img.meta

In [76]:
#eda
#getting this error: ValueError: <COMPRESSION.LZW: 5> requires the 'imagecodecs' package

#img = tifffile.imread('/content/drive/MyDrive/capstone/Satellite_Imagery/Satellite_Images_Tiled/36.tif')[0:148, 0:188, :]
#img2 = tifffile.imread('/content/drive/MyDrive/capstone/Satellite_Imagery_Satellite_Images_Tiled/37.tif')[0:148, 0:188, :]

In [75]:
#img1
np.shape(img1)

(4, 149, 189)

In [79]:
# eda
test = []
test.append(img1[:, 0:148, 0:188])
test.append(img2[:, 0:148, 0:188])
print(np.shape(test))
test = np.stack(test)
print(np.shape(test))

(2, 4, 148, 188)
(2, 4, 148, 188)


In [84]:
def preprocess_data_part1(IMAGE_PATH):
    """ Generate lists of images and labelsbased on temp_no_refer and temp_refer lists
    
    Params:
    -------
    IMAGE_PATH (str): path to directory with images.
    
    Returns:
    --------
    images_mini  (np.ndarray): Images of shape (N, 149 3)
    """
    
    data_mini = []
    
    for id in X_train['Tile_ID']:    
                    
        # read image and store as matrix            
        # Index at the end makes all images the same size (they sometimes differ by 1 pixel)
        with rio.open(IMAGE_PATH + str(id) + '.tif') as img :

            image = img.read()[0:148, 0:188, :]
        
        #image = tifffile.imread(IMAGE_PATH + str(id) + '.tif')[0:148, 0:188, :]
            
        # append to images
        data_mini.append(image)
 
    # stack images and trasnform to array
    images_mini = np.stack(data_mini)
    
    return images_mini

In [85]:
# THERE IS ONE FILE IN THE "SATELLITE IMAGES TILED" FOLDER THAT IS NOT A TIF IMAGE, DELETE THIS FILE
images_mini = preprocess_data_part1(IMAGE_PATH)

ValueError: ignored

In [ ]:
np.shape(images_mini)

In [ ]:
# RESNET 18 TAKEN FROM https://github.com/jimmyyhwu/resnet18-tf2/blob/master/resnet.py
# HELPS RUN MODEL BELOW

kaiming_normal = keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='untruncated_normal')

def conv3x3(x, out_planes, stride=1, name=None):
    x = layers.ZeroPadding2D(padding=1, name=f'{name}_pad')(x)
    return layers.Conv2D(filters=out_planes, kernel_size=3, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=name)(x)

def basic_block(x, planes, stride=1, downsample=None, name=None):
    identity = x

    out = conv3x3(x, planes, stride=stride, name=f'{name}.conv1')
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn1')(out)
    out = layers.ReLU(name=f'{name}.relu1')(out)

    out = conv3x3(out, planes, name=f'{name}.conv2')
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn2')(out)

    if downsample is not None:
        for layer in downsample:
            identity = layer(identity)

    out = layers.Add(name=f'{name}.add')([identity, out])
    out = layers.ReLU(name=f'{name}.relu2')(out)

    return out

def make_layer(x, planes, blocks, stride=1, name=None):
    downsample = None
    inplanes = x.shape[3]
    if stride != 1 or inplanes != planes:
        downsample = [
            layers.Conv2D(filters=planes, kernel_size=1, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=f'{name}.0.downsample.0'),
            layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.0.downsample.1'),
        ]

    x = basic_block(x, planes, stride, downsample, name=f'{name}.0')
    for i in range(1, blocks):
        x = basic_block(x, planes, name=f'{name}.{i}')

    return x

def resnet(x, blocks_per_layer, num_classes=1000):
    x = layers.ZeroPadding2D(padding=3, name='conv1_pad')(x)
    x = layers.Conv2D(filters=64, kernel_size=7, strides=2, use_bias=False, kernel_initializer=kaiming_normal, name='conv1')(x)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='bn1')(x)
    x = layers.ReLU(name='relu1')(x)
    x = layers.ZeroPadding2D(padding=1, name='maxpool_pad')(x)
    x = layers.MaxPool2D(pool_size=3, strides=2, name='maxpool')(x)

    x = make_layer(x, 64, blocks_per_layer[0], name='layer1')
    x = make_layer(x, 128, blocks_per_layer[1], stride=2, name='layer2')
    x = make_layer(x, 256, blocks_per_layer[2], stride=2, name='layer3')
    x = make_layer(x, 512, blocks_per_layer[3], stride=2, name='layer4')

    x = layers.GlobalAveragePooling2D(name='avgpool')(x)
    initializer = keras.initializers.RandomUniform(-1.0 / math.sqrt(512), 1.0 / math.sqrt(512))
    x = layers.Dense(units=num_classes, kernel_initializer=initializer, bias_initializer=initializer, name='fc')(x)

    return x

def resnet18(x, **kwargs):
    return resnet(x, [2, 2, 2, 2], **kwargs)

def resnet34(x, **kwargs):
    return resnet(x, [3, 4, 6, 3], **kwargs)

**RANDOM FOREST DECISION TREE**

https://www.dlology.com/blog/simple-guide-on-how-to-generate-roc-plot-for-keras-classifier/

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#regr = RandomForestRegressor(n_estimators = 100, random_state = 0)
#regr.fit(X_train, y_train.values.ravel())

#y_pred_rf = regr.predict(X_test)[:]

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#from yellowbrick.model_selection import learning_curve

#rfc = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
#print(learning_curve(rfc, X, y, cv=10, scoring='accuracy'))

In [ ]:
#dataset = tiles

#tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label="label")

In [86]:
model = tfdf.keras.RandomForestModel()

history = model.fit(
    np.asarray(X_train).astype('float32'),
    y_train,
    epochs=1,
    verbose=1,
    validation_split = 0.2)

Use /tmp/tmp7vd8mq3g as temporary training directory


Reading training dataset...


Training dataset read in 0:00:00.589511. Found 2617 examples.
Training model...
Model trained in 0:00:04.649620
Compiling model...


Model compiled.


In [87]:
print(model.summary())

Model: "random_forest_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	data:0.0
	data:0.1
	data:0.10
	data:0.11
	data:0.2
	data:0.3
	data:0.4
	data:0.5
	data:0.6
	data:0.7
	data:0.8
	data:0.9

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "data:0.11"  0.416558 ################
    2.  "data:0.7"  0.238669 ######
    3. "data:0.10"  0.192846 ####
    4.  "data:0.0"  0.171230 ###
    5.  "data:0.5"  0.133832 #
    6.  "data:0.4"  0.130739 #
    7.  "data:0.2"  0.130187 #
    8.  "data:0.3"  0.128937 #
    9.  "data:0.1"  0.124725 
   10.  "data:0.6"  0.123617 
   11.  "data:0.8"  0.105894 
   12.  "data:0.9"  0.105289 

Variable Importance: NUM_AS_ROOT:
    1. "data:0.11" 128.00

**Gradient Boosted Decision Tree**

In [88]:
model = tfdf.keras.GradientBoostedTreesModel()

history = model.fit(
    np.asarray(X_train).astype('float32'),
    y_train,
    epochs=1,
    verbose=1,
    validation_split = 0.2)

Use /tmp/tmpvdzme_y8 as temporary training directory


Reading training dataset...


Training dataset read in 0:00:00.423521. Found 2617 examples.
Training model...
Model trained in 0:00:04.265358
Compiling model...


Model compiled.


In [89]:
print(model.summary())

Model: "gradient_boosted_trees_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (12):
	data:0.0
	data:0.1
	data:0.10
	data:0.11
	data:0.2
	data:0.3
	data:0.4
	data:0.5
	data:0.6
	data:0.7
	data:0.8
	data:0.9

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "data:0.11"  0.707534 ################
    2.  "data:0.7"  0.247596 #
    3. "data:0.10"  0.228859 #
    4.  "data:0.0"  0.220313 #
    5.  "data:0.5"  0.190502 
    6.  "data:0.2"  0.190416 
    7.  "data:0.4"  0.188565 
    8.  "data:0.9"  0.185540 
    9.  "data:0.8"  0.184934 
   10.  "data:0.6"  0.184692 
   11.  "data:0.1"  0.184309 
   12.  "data:0.3"  0.183709 

Variable Importance: NUM_AS_ROOT:
    1. "data:0.11" 37